In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [5]:
#give credit to Theo Viel from https://www.kaggle.com/theoviel/improve-your-score-with-some-text-preprocessing
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
glove = './input/embeddings/glove.840B.300d/glove.840B.300d.txt'
paragram =  './input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
wiki_news = './input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'

In [6]:
train.sample(10)

,qid,question_text,target
853896,a74d7ae7bbfd1da83da0,What is a pilot test in research?,0
369167,48625812ea248aa7635a,How long do you have before a Snapchat streak ...,0
942438,b8b1c5c92c104616796f,What are some tricks for mixing and mastering ...,0
1226864,f071f84dacb57b43b4b3,What would be the cost and estimated time of c...,0
1161391,e38fd4f4ee46cad5e565,Has being on Quora changed your conception of ...,0
1218571,eed4396f2e4bcfde0bde,What is Skandar Keynes known for?,0
25324,04f85059238fb6ea61c1,Is water really wet? Or is the internet losing...,0
182869,23bf13a2a78dfaca969a,Which are the Mark awarding faculties for IP i...,0
290597,38e96e18fddaa4825098,If the communist China decides to invade Taiwa...,0
529681,67b5a82082f28cd3993e,Where would Minato or go or what would happen ...,0


In [7]:
test.sample(10)

,qid,question_text
50830,e6803496deb741f271d4,What's the best YouTube video to watch to lear...
39548,b2b26392aab2c2ee3d53,Who's the best mathematics teacher for general...
36861,a6320268cd11be86b9e4,"Do you believe in brainwashing, and if so is t..."
24870,70617568a7f635de7ab4,Which chapters contribute the most in ICSE bio...
31829,8f950a9fa5b2a7de6b2a,Why has Quora sacrificed its valued intellectu...
47800,d8586ab323d07be72d7e,How should I prepare for CGL 2018?
47181,d57bd0e7d95288dae9ce,What is the strongest spell that Zatana got?
25539,7361386ab4bf8f162d57,What does it mean on Spokeo when it says sourc...
33192,95bc3ef0726aabc118ee,Can anyone become travel show host at the age ...
17105,4d555457c9faf3663921,How many Bangladeshi lives in Germany?


In [3]:
import numpy as np
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == './input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

In [15]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [16]:
import operator
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [18]:
def add_lower(embedding, vocab):
    count = 0
    for word in vocab:
        if word in embedding and word.lower() not in embedding:  
            embedding[word.lower()] = embedding[word]
            count += 1
    print(f"Added {count} words to embedding")

In [19]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [20]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [21]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [22]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [23]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

In [24]:
def mydatacleanning(df):
    df['lowered_question'] = df['question_text'].apply(lambda x: x.lower())
    df['treated_question'] = df['lowered_question'].apply(lambda x: clean_contractions(x, contraction_mapping))
    df['treated_question'] = df['treated_question'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
    vocab = build_vocab(df['treated_question'])
    print("Glove : ")
    embed_glove = load_embed(glove)
    oov_glove = check_coverage(vocab, embed_glove)
    print("Paragram : ")
    embed_paragram = load_embed(paragram)
    oov_paragram = check_coverage(vocab, embed_paragram)
    print("FastText : ")
    embed_fasttext = load_embed(wiki_news)
    oov_fasttext = check_coverage(vocab, embed_fasttext)
    return df

In [32]:
from sklearn.model_selection import train_test_split
train_df_original = pd.read_csv("../input/train.csv")



In [33]:
train_df_clean=mydatacleanning(train_df_original)
train_df, val_df = train_test_split(train_df_clean, test_size=0.1)

Glove : 
Found embeddings for 63.34% of vocab
Found embeddings for  99.38% of all text
Paragram : 
Found embeddings for 73.99% of vocab
Found embeddings for  99.63% of all text
FastText : 
Found embeddings for 48.17% of vocab
Found embeddings for  98.80% of all text


In [34]:
train_df.sample(2)
val_df.sample(2)

,qid,question_text,target,lowered_question,treated_question
834855,a399760b6ff8d91d45ee,Has anyone read Talon of God by Wesley Snipes ...,0,has anyone read talon of god by wesley snipes ...,has anyone read talon of god by wesley snipes ...
332052,4117d020b2cfc0f3bdfe,Are there any browsers with dual-engine WebKit...,0,are there any browsers with dual-engine webkit...,are there any browsers with dual - engine we...


In [35]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [36]:
# embdedding setup
# Source https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
# Give credit to Miha Skalic from https://www.kaggle.com/mihaskalic/lstm-is-all-you-need-well-maybe-embeddings-also
embeddings_index = {}
f = open('./input/embeddings/glove.840B.300d/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [03:15, 11245.57it/s]

Found 2196016 word vectors.


In [37]:
# Convert values to embeddings
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:20]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (20 - len(embeds))
    return np.array(embeds)

val_vects = np.array([text_to_array(X_text) for X_text in tqdm(val_df["treated_question"][:3000])])
val_y = np.array(val_df["target"][:3000])

100%|██████████| 3000/3000 [00:00<00:00, 16637.95it/s]


In [38]:
# Data providers
batch_size = 64

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

In [39]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional

In [40]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
                        input_shape=(20, 300)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [41]:
mg = batch_gen(train_df)
model.fit_generator(mg, epochs=30,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

Epoch 1/30
1000/1000 [==============================] - 43s 43ms/step - loss: 0.1392 - acc: 0.9474 - val_loss: 0.1452 - val_acc: 0.9383
Epoch 2/30
1000/1000 [==============================] - 35s 35ms/step - loss: 0.1235 - acc: 0.9514 - val_loss: 0.1355 - val_acc: 0.9457
Epoch 3/30
1000/1000 [==============================] - 35s 35ms/step - loss: 0.1203 - acc: 0.9530 - val_loss: 0.1358 - val_acc: 0.9467
Epoch 4/30
1000/1000 [==============================] - 35s 35ms/step - loss: 0.1203 - acc: 0.9543 - val_loss: 0.1318 - val_acc: 0.9487
Epoch 5/30
1000/1000 [==============================] - 35s 35ms/step - loss: 0.1160 - acc: 0.9546 - val_loss: 0.1335 - val_acc: 0.9500
Epoch 6/30
1000/1000 [==============================] - 35s 35ms/step - loss: 0.1121 - acc: 0.9562 - val_loss: 0.1274 - val_acc: 0.9550
Epoch 7/30
1000/1000 [==============================] - 35s 35ms/step - loss: 0.1119 - acc: 0.9550 - val_loss: 0.1261 - val_acc: 0.9537
Epoch 8/30
1000/1000 [==========================

In [43]:
# prediction part
batch_size = 256
def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr

test_df = pd.read_csv("./input/test.csv")


all_preds = []
for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())

221it [00:17, 12.74it/s]


In [46]:
y_te = (np.array(all_preds) > 0.5).astype(np.int)

submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.sample(2)


,qid,prediction
51495,e971bace7107d3da76d2,0
31865,8fc9588e0ac80af5c294,0
